In [48]:
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
import glob
import os
import numpy as np
import math

In [49]:
iceCount = list()
averageTemp = list()
particleID = list()
iceVolume=list()

In [50]:
def createIceParticleIDIceVolume(data):
#     print(data)
    dataSortedByIce = data[data.iceLabel == 1]
#     print(dataSortedByIce)
    particleID.append(list(dataSortedByIce.origId))
    totalVolume = 0.0
    for index, row in dataSortedByIce.iterrows():
        totalVolume += (1/6)*math.pi*math.pow(row['d'],3)
    iceVolume.append(totalVolume)

In [51]:
path = 'contrails raw data/contrails 3/'
for filename in glob.glob(os.path.join(path, '*.csv')):
   with open(os.path.join(os.getcwd(), filename), 'r') as f: # open in readonly mode
      # do your stuff
        contrails = pd.read_csv(f)
        createIceParticleIDIceVolume(contrails)
        valueCountIce = list(contrails.iceLabel.value_counts())
        averageTemp.append(round(contrails["T"].mean(),2))
        iceCount.append(valueCountIce)
        

In [52]:
newIceParticles = list()
# adding the first time steps
newIceParticles.append(len(particleID[0]))
print(newIceParticles)
for i in range(0, len(particleID)-1):
    count = 0
    for j in range(0, len(particleID[i])):
#         print(count)
        if(particleID[i][j] in particleID[i+1]):
            count = count + 1
    newIceParticles.append(len(particleID[i+1]) - count)
        

[5]


In [53]:
newIceParticles

[5, 149, 405, 403, 424, 413, 442, 427, 391, 436, 444, 427, 424, 446, 390]

In [54]:
iceCount

[[419, 5],
 [695, 154],
 [715, 559],
 [961, 740],
 [1384, 741],
 [1465, 752],
 [1481, 730],
 [1483, 727],
 [1441, 761],
 [1452, 755],
 [1463, 738],
 [1460, 746],
 [1482, 749],
 [1499, 730],
 [1456, 762]]

In [55]:
averageTemp

[601.78,
 456.71,
 385.37,
 348.25,
 323.24,
 318.09,
 317.22,
 318.84,
 319.95,
 320.23,
 318.31,
 319.69,
 318.65,
 318.17,
 318.81]

In [56]:
iceVolume

[2.697247279553876e-15,
 9.452638269150322e-14,
 3.4509082539614875e-13,
 6.047881255859209e-13,
 8.788189126181662e-13,
 9.273318102257912e-13,
 9.331270779909389e-13,
 9.323617248046148e-13,
 9.090918608838728e-13,
 9.173769736131729e-13,
 9.245584994355803e-13,
 9.24286926822431e-13,
 9.33921954472019e-13,
 9.380254614442435e-13,
 9.088200500321533e-13]

In [57]:
extractedData = pd.DataFrame(iceCount, columns=["noIce","Ice"])
total = list()
percentage = list()
timesteps = np.round(np.arange(0.06,0.2,0.01),2).tolist()
# timesteps
for index, row in extractedData.iterrows():
    sumParticle = row["noIce"] + row["Ice"]
    total.append(round(sumParticle))
    percentage.append(round((row["Ice"]/sumParticle)*100,2))
    
extractedData.insert(0, "Timesteps", timesteps, True)
extractedData.insert(1, "TotalParticles", total, True)
extractedData["NewIce"] = newIceParticles
extractedData["TotalIcePercentage"] = percentage
extractedData["Temp"] = averageTemp
extractedData["IceVolume"] = iceVolume


In [58]:
remainingIceParticles = list()
newIcePercentage = list()
remainingIceParticles.append(total[0])
newIcePercentage.append(round((iceCount[0][1]/total[0])*100,2))
for t in range(1, len(total)):
    temp = total[t] - iceCount[t-1][1]
    remainingIceParticles.append(temp)
    newIcePercentage.append(round((newIceParticles[t]/temp)*100,2))
    

extractedData["TotalNonIce"] = remainingIceParticles
extractedData["newIcePercentage"] = newIcePercentage


In [59]:
 
extractedData

,Timesteps,TotalParticles,noIce,Ice,NewIce,TotalIcePercentage,Temp,IceVolume,TotalNonIce,newIcePercentage
0,0.06,424,419,5,5,1.18,601.78,2.697247e-15,424,1.18
1,0.07,849,695,154,149,18.14,456.71,9.452638e-14,844,17.65
2,0.08,1274,715,559,405,43.88,385.37,3.450908e-13,1120,36.16
3,0.09,1701,961,740,403,43.50,348.25,6.047881e-13,1142,35.29
4,0.10,2125,1384,741,424,34.87,323.24,8.788189e-13,1385,30.61
5,0.11,2217,1465,752,413,33.92,318.09,9.273318e-13,1476,27.98
6,0.12,2211,1481,730,442,33.02,317.22,9.331271e-13,1459,30.29
7,0.13,2210,1483,727,427,32.90,318.84,9.323617e-13,1480,28.85
8,0.14,2202,1441,761,391,34.56,319.95,9.090919e-13,1475,26.51
9,0.15,2207,1452,755,436,34.21,320.23,9.173770e-13,1446,30.15


In [60]:
extractedData.to_csv ('Extracted Data/contrails 3.csv', index = False, header=True)